In [ ]:
import torch
import matplotlib.pyplot as plt
from pssr.data import ImageDataset
from pssr.models import ResUNet
from pssr.loss import SSIMLoss
from pssr.train import fit_paired

In [ ]:
lr = 1e-4
win_size = 11

lr_scale = 4

In [ ]:
dataset = ImageDataset("data/EM_hr_1_10")

In [ ]:
model = ResUNet(
    channels=1,
    hidden=[64, 128, 256, 512, 1024],
    scale=lr_scale,
    depth=2,
)

In [ ]:
batch_size = 16
device = "cuda" if torch.cuda.is_available() else "cpu"

kwargs = dict(
    num_workers = 4,
    pin_memory = True,
)

In [ ]:
loss_fn = SSIMLoss(mix=.6, win_size=win_size, ms=True)
optim = torch.optim.AdamW(model.parameters(), lr=lr)

losses = fit_paired(
    model=model,
    dataset=dataset,
    epochs=1,
    batch_size=batch_size,
    loss_fn=loss_fn,
    optim=optim,
    device=device,
    dataloader_kwargs=kwargs,
    log_frequency=50,
)

In [ ]:
plt.plot(losses)

In [ ]:
torch.save(model.state_dict(), "model.pth")